In [1]:
import numpy as np
import itertools


from datetime import datetime, timedelta

import sklearn.metrics as metrics


import pandas as pd

import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")

# Utils Functions (helper functions for the time series classes) 

In [2]:
def generate_prophet_time_frame(start_date , forecast_steps=None,end_date=None):
    
    if isinstance(start_date,str):
        start_date=datetime.strptime(start_date,'%Y/%m/%d')
    
    if forecast_steps is None and end_date is not None:
        if isinstance(end_date,str):
            end_date=datetime.strptime(end_date,'%Y/%m/%d')
    else :
        end_date=start_date + timedelta(days = forecast_steps-1)
    
    return pd.DataFrame(pd.date_range(start_date,end_date),columns=["ds"])
    

In [3]:
'''Functions that initilize an empty dataframe to put the metadata about the 
1- Model ( model id , last training time)
2-Forecasts (model_id , predicted values , confidence intervals)
'''

# Parameters= {"vxp_client_id": 1 , 
#              "metric":"DAU",
#             "game":'All',
#             "platform" : 'All'}


model_db_columns=["model_id","model_path","model_type"
                  "model_training_day","model_train_starttime","model_train_endtime",
                  "model_test_starttime","model_test_endtime",
                  "training_type","model_name",
                  "model_train_MAE","model_test_MAE" ,                  
                  "vxp_client_id" , "metric" , "game", "platform"
                 ]


forecast_columns=["ds","yhat","yhat_lower","yhat_upper","model_id"]


def initialize_model_db(model_db_columns=model_db_columns):    
#     model_db_columns=["model_id","model_train_starttime","model_train_endtime","model_test_starttime","model_test_endtime",
#             "training_type","model_training_day","model_train_MAE","model_test_MAE"]
    return pd.DataFrame([],columns=model_db_columns)
    
    
def initialize_forecast_db(forecast_columns=forecast_columns):
#     forecast_columns=["ds","yhat","yhat_lower","yhat_upper","model_id"]
    return pd.DataFrame([],columns=forecast_columns)




def save_pandas_dataframe(df_new,df_old=None):        
    '''Function that takes in two dataframes ,append one to the other . Useful for 
    -Appending new models to the existing model DB
    -Appending new forecasts to the existing forecast DB
    '''
    if df_old is None :
        return df_new
    else:
        if set(list(df_new.columns.values))==set(list(df_old.columns.values)):
            return df_old.append(df_new)
        else :
            raise Exception("Cant add Dataframes")    

def next_timestamp(start_date_string):
     x=datetime.strptime(start_date_string,'%Y/%m/%d') + timedelta(days =1)
     return x.strftime('%Y/%m/%d')

def nonetype_float(x):
    if x is None :
        return np.nan
    else :
        return x

def nonetype_timestamp(x):
    if x is None :
        return ''
    else :
        return x.strftime("%Y-%m-%d")
        

In [4]:
#Define a function that takes in time of running and last fit date and returns all times between 
def date_between(model_fit_date):
    return generate_prophet_time_frame(start_date=model_fit_date,end_date=datetime.now())

#Here we investigate if something uprupt happends so we decide wether to fit , hypertune , or just forecast

# Workspace